<a href="https://colab.research.google.com/github/gabrielsimas/rio-viagens-onibus-analytics/blob/feature%2Fcriacao-ambiente/ARTIGO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**IMPORTANTE**: ***Executar esse trecho de código para baixar o repositório e alimentar as imagens desse arquivo.***

# 🚌 MVP Engenharia de Dados - O Rio de Janeiro e os Ônibus: Entendendo o Caos!
## ***Nome:*** **Luís Gabriel Nascimento Simas**
## ***Matrícula:*** **4052025000943**

## **Objetivo**

### O Rio de Janeiro, mundialmente conhecido por suas belezas naturais, enfrenta um desafio urbano de proporções continentais: a mobilidade pública. O sistema de transporte rodoviário, vital para o funcionamento da metrópole, é frequentemente associado à palavra "**Caos**".
### A população carioca convive diariamente com incertezas: tempos de espera imprevisíveis, frota envelhecida, desconforto térmico em uma cidade tropical e falhas na cobertura do serviço, especialmente em horários noturnos. Embora iniciativas como o BRT (*Bus Rapid Transit*) e o BRS (*Bus Rapid System*) tenham sido implementadas para mitigar a lentidão, a percepção de qualidade do serviço permanece aquém do ideal.
### Não se trata apenas de desconforto, mas de um problema sistêmico que envolve a gestão de consórcios privados, a fiscalização do poder público e o impacto direto na qualidade de vida e produtividade do cidadão.
### Se por um lado o cenário físico é caótico, o cenário digital oferece uma oportunidade de organização e entendimento. A Prefeitura da Cidade do Rio de Janeiro (PCRJ), através do portal *[data.rio](https://data.rio)*, disponibiliza um vasto repositório de dados abertos em vários formatos e alguns até como tabelas no **Google** *BigQuery*.
### Esses dados incluem registros detalhados de viagens (baseados em bilhetagem e telemetria consolidada), cadastro técnico da frota, dados meteorológicos do Alerta Rio e registros de reclamações da central 1746.

### Para guiar o desenvolvimento do pipeline de dados, definimos as seguintes questões-chave que esse trabalho deseja responder.


- ### **Dinâmica de Viagens e Demanda**
    - #### 1. Em quais dias e faixas horárias o volume de viagens realizadas atinge seu pico?
    - #### 2. Existe uma redução sensível na oferta de viagens nos finais de semana comparados aos dias úteis?
    - #### 3. Qual é a duração média das viagens por Consórcio? Existe discrepância significativa entre eles?

- ### **Consistência Operacional**
    - ### 4. O tempo de viagem nas principais linhas é consistente ou apresenta alto desvio padrão (imprevisibilidade)?
    - ### 5. Em dias classificados como "Chuvosos" (precipitação média acima de um limiar), observa-se aumento no tempo médio de deslocamento?
    - ### 6. A chuva influencia na quantidade total de viagens realizadas (redução da oferta ou demanda)?

### **Escopo temporal**
### A análise compreenderá o período de **01/01/2024** a **19/12/2025**, permitindo uma visão longitudinal que abrange sazonalidades, dias úteis, feriados e variações climáticas ao longo de quase dois anos completos.
### Antes de começar a explicar o mecanismo de coleta de dados que foi utilizado, é importante mostrar em qual arquitetura nosso trabalho está sendo desenvolvido/implementado.


## 🏛️ **Arquitetura da Solução**: Data Lakehouse Híbrido (VPS + GCP)

### Esta seção detalha os componentes tecnológicos do MVP. A arquitetura adota uma estratégia **Híbrida**, combinando a otimização de recursos de um VPS (Virtual Private Server) com a escalabilidade de armazenamento da Nuvem Pública (Google Cloud).

---

### **1. Provisionamento de Infraestrutura (IaC)**

### A infraestrutura na nuvem não foi criada manualmente. Todo o ambiente (Buckets, Permissões, APIs) foi definido via **Terraform**, garantindo que o ambiente seja reprodutível, auditável e imune a erros humanos.

### 🏗️ **Terraform Workflow**
### **Papel:** Provisionamento de Nuvem.
* ### **Função:** Cria e gerencia os recursos na GCP (Landing, Bronze, Silver, Gold) de forma declarativa.
* ### **Ciclo de Vida:** O deploy seguiu as fases de *Bootstrap*, *Validation*, *Plan* e *Apply*.

<details>

<summary>
<strong>
Clique para expandir: Evidências da Execução do Terraform
</strong>
</summary>
<br>

<h2 style="margin-bottom:0">1. Preparação do Ambiente (Bootstrap)</h2>
<img src="https://drive.google.com/uc?id=1Iq7M2xjMHbjlrEMHYy36VCb8tu1WXJQZ" width="100%" alt="Script Inicial">
<p><em>Setup inicial das variáveis e autenticação no Google Cloud Shell.</em></p>
<hr>

<h2 style="margin-bottom:0">2. Validação e Planejamento (Plan)</h2>
<img src="https://drive.google.com/uc?id=1tdGPpca2xtP4oKW0zmfELg5opni451Ao" width="100%" alt="Terraform Validate">
<p><em>Validando a sintaxe dos arquivos .tf para garantir qualidade do código.</em></p>
<br>
<img src="https://drive.google.com/uc?id=1PIAl4zRuyEZYhEJJVuPNaItXQIO99RUm" width="100%" alt="Terraform Plan">
<p><em>Terraform Plan: O sistema calcula o "estado futuro" e prevê as mudanças antes de tocar na nuvem.</em></p>
<hr>

<h2 style="margin-bottom:0">3. Aplicação e Resultado (Apply)</h2>
<img src="https://drive.google.com/uc?id=1faj6oTVGA2jjHIyjzZ5EzMl1kRyUxE8S" width="100%" alt="Terraform Apply Sucesso">
<p><em>Terraform Apply: Criação automatizada dos recursos (Buckets, Service Accounts).</em></p>
<br>
<img src="https://drive.google.com/uc?id=1JaqcI2S5XlZiu6WKHt6Ibn3nzoiGHKzK">
<p><em>Resultado Final: O ambiente pronto no console da Google Cloud, refletindo exatamente o código.</em></p>

</details>

---

### **2. Servidor de Processamento (Compute On-Premise)**

###Para viabilizar o uso de ferramentas corporativas (Dremio/Java) em um hardware limitado, foi realizado um trabalho avançado de **Engenharia de Sistemas** no VPS.

### 🖥️ **Especificações e Otimização**
* ### **Hardware:** VPS com processador AMD EPYC (4 vCPUs) e **6GB de RAM**.
* ### **Estratégia de Swap:** Devido à limitação de memória física, configurou-se uma área de swap de **22GB+** via terminal Linux. Isso permite que o *Heap* do Java (Dremio) e os processos do Airflow rodem simultaneamente sem causar *Out Of Memory (OOM)*.

### 🐧 **Hardening e Segurança**
### O ambiente segue o princípio do menor privilégio, evitando a execução de containers como `root` e utilizando chaves SSH para acesso.

<details>
<summary><strong>🎥 Clique para expandir: Engenharia de Sistemas (Linux Tuning)</strong></summary>
<br>

<h2 style="margin-bottom:0">1. Acesso Seguro e Criação de Usuário</h2>
<img src="https://drive.google.com/uc?id=1KVESwnVCpXmFbDKCJSFuEQzm7-cp6eB4" width="100%" alt="Acesso SSH">
<p><em>Acesso remoto seguro via SSH com chaves criptografadas (sem senha).</em></p>
<br>
<img src="https://drive.google.com/uc?id=1-ruL8NLRbhD9FZ333c7H0A2sVInfK411" width="100%" alt="User Setup">
<p><em>Segurança: Criação de usuário dedicado para evitar execução de containers como root.</em></p>
<hr>

<h2 style="margin-bottom:0">2. Expansão de Memória (Swap Strategy)</h2>
<img src="https://drive.google.com/uc?id=1jy_hFiKoHqKumGM-0ZMaYvaQ8GZdCEdN" width="100%" alt="Swap Creation">
<p><em>Engenharia: Criação manual de arquivo de Swap e ajuste de swappiness para suportar carga pesada.</em></p>

</details>

---

## **3. Orquestração e Ingestão**

### 🐳 **Docker Compose**
### **Papel:** Orquestração de Containers.
* ### **Função:** Sobe e conecta os serviços do Airflow, Dremio, Nessie e Postgres em uma rede interna isolada, garantindo portabilidade.

Aqui, temos uma imagem de nosso ambiente com os containers utilizados nessa solução operacionais, prints das aplicações Lazydocker e Portainer respectivamente.

![Containers em pé com LazyDocker](https://drive.google.com/uc?id=1tQUFttV93jUKBNndXEnMsOfUpfpn0s_Q)

![Container em pé com Portainer](https://drive.google.com/uc?id=14HgdV4boaHVyKzcbDsesDmg0peh_aF2J)


### 🌪️ **Apache Airflow**
### **Papel:** Orquestrador Geral (O "Maestro").
* ### **Função:** Gerencia todo o pipeline de dados. Monitora o Google Drive, dispara os workers de ingestão e aciona o **dbt** para as transformações.
* ### **Otimização:** Configurado com `LocalExecutor` para paralelizar tarefas nos 4 núcleos da CPU.

![Tela do Airflow com DAGs](https://drive.google.com/uc?id=1AYCCyarwBsubUZ0F-i33rFUd5FrVgIHD)

### 🦆 **DuckDB (Worker)**
### **Papel:** Processamento Leve de Ingestão.
* ### **Função:** Banco analítico embutido (*in-process*) que roda dentro do worker do Airflow.
* ### **No MVP:** Lê arquivos CSV brutos da camada Landing, limpa e converte para **Parquet** na camada Bronze. Sua eficiência permite processar gigabytes de dados com pouca RAM.

### ☁️ **Google Cloud Storage (GCS)**
### **Papel:** Armazenamento de Objetos (Storage Layer).
* ### **Camadas:**
    * ### **Landing:** Dados brutos (CSV).
    * ### **Bronze:** Dados históricos convertidos para Parquet.
    * ### **Silver/Gold:** Dados de negócio no formato **Apache Iceberg**.

---

## 4. **O Motor do Lakehouse (Engine & Transformação)**

### 💎 **Dremio**
### **Papel:** SQL Lakehouse Engine.
* ### **Função:** Motor de consulta que permite executar SQL diretamente sobre o Data Lake.
* ### **Performance:** Tunado com **Heap de 2.5GB** e **Direct Memory de 1GB** para operar no limite do hardware, delegando carga excedente para o Swap.

### 🧊 **Apache Iceberg**
### **Papel:** Formato de Tabela Aberto.
* ### **Função:** Camada de abstração que traz transações ACID e *Time Travel* para o Data Lake, permitindo updates e deletes seguros em arquivos Parquet.

### 🛠️ **dbt (data build tool)**
### **Papel:** Lógica de Transformação.
* ### **Função:** Compila as regras de negócio (SQL) e envia para o Dremio executar. Transforma dados da Bronze em tabelas Silver (limpas) e Gold (agregadas).

---

## **5. Governança**

### 🦕 **Project Nessie**
### **Papel:** Catálogo de Dados (Git-for-Data).
* ### **Função:** Gerencia as versões das tabelas Iceberg.
* ### **Recurso:** Permite criar *Branches* de dados para testes isolados antes de fazer *Merge* na produção, similar ao Git para código.

### **O que é a Arquitetura Moderna de Dados** ?
### É uma arquitetura **flexível**, **escalável** e **baseada na nuvem**. Ela surge dos modelos tradicionais para suportar o volume e a velocidade dos dados atuais, integrando **Big Data**, **IA** e **governança**. Ela tem como características principais:
- ### **Nuvem** e **Tempo Real**: Prioriza a elasticidade (AWS, Azure, GCP) e a disponibilidade imediata dos dados para tomadas de decisão rápidas. Neste MVP, vamos utilizar a GCP como nuvem principal, mas a solução que vamos implementar é totalmente agnóstica em questões de hospedagem.
- ### **Dados como Produto**: O gerenciamento é focado no valor que o dado gera para o negócio.
- ### **Governança e Segurança**:  Implementadas na base (considerando LGPD), com forte gestão de metadados e interoperabilidade via APIs.
### As abordagens mais populares dessa arquitetura são:
- ### **Data Lakehouse**: O melhor dos dois mundos (flexibilidade do Lake + confiabilidade do Warehouse). ***Será a arquitetura que utilizaremos nesse MVP.***
- ### **Data Mesh**: Descentraliza a propriedade dos dados por domínios de negócio.
- ### **Modern Data Stack (MDS)**: Orquestração de diversas ferramentas especializadas.

### Tem como principal objetivo unificar dados, análises e cargas de trabalho de Inteligência Artificial de forma **eficiente, confiável e escalável**.
### ***Neste MVP, vamos focar no uso do Data Lakehouse com a Arquitetura Medalhão***
### Antes da seção sobre a infraestrutura que será utilizada, passaremos rapidamente sobre os significados e propósitos do **Data Lakehouse** e **Arquitetura Medalhão**


### **Data Lakehouse**

<center>
<img src="https://drive.google.com/uc?id=1jV69fy2AMAd4YZx0BACaQPwHQLkCJNH1" alt="Girl in a jacket" width="400">
<center />

### O **Data Lakehouse** representa uma evolução significativa na arquitetura de dados, surgindo para resolver a dicotomia histórica entre Data Warehouses e Data Lakes.
### A premissa básica é simples, mas poderosa: **implementar funcionalidades de gestão de dados (típicas de Warehouses) diretamente sobre o armazenamento de baixo custo e flexível (típico de Lakes)**.


### **O Problema Original (Silos de Dados)**
Para entender o Lakehouse, precisamos olhar para o que existia antes:
- ### **Data Warehouse (DW)**: Ótimo para dados estruturados, SQL rápido e confiabilidade (ACID). **Entretanto**, é caro, rígido e péssimo para lidar com dados não estruturados (vídeo, áudio, logs brutos) ou ML (Machine Learning).
- ### **Data Lake**: Ótimo para armazenamento (arquivos brutos) e flexível para Ciência de Dados. **Entretanto**, tende a virar um "Data Swamp" (pântano de dados) sem governança de Dados, não suporta transações complexas e tem performance pobre para BI tradicional.

### Muitas empresas mantinham os dois sistemas duplicados, criando complexidade e alto custo de manutenção.

### **O Conceito do Lakehouse**
### O Data Lakehouse elimina a necessidade de mover os dados do Lake para o Warehouse para que sejam úteis. Ele adiciona uma camada de metadados e governança sobre os arquivos brutos armazenados no Lake.

### **Principais Pilares Técnicos:**
- ### **Transações ACID**: Esta é a grande virada de chave. O Lakehouse permite que múltiplos usuários leiam e escrevam dados simultaneamente sem corromper a base. Se uma carga de dados falha no meio, nada é salvo (atomicidade), garantindo que os dados estejam sempre consistentes.
- ### **Schema Enforcement (Imposição de Esquema)**: O sistema previne que dados "sujos" ou fora do padrão sejam inseridos nas tabelas, garantindo a qualidade necessária para relatórios de BI.
- ### **Formatos Abertos**: Diferente dos DWs tradicionais que usam formatos proprietários (fechados), o Lakehouse armazena dados em formatos open-source como **Parquet** ou **ORC**. Isso evita o *vendor lock-in* (ficar preso a um fornecedor).
- ### **Suporte Unificado (BI + IA)**: Cientistas de dados podem acessar os arquivos diretamente para treinar modelos de IA, enquanto Analistas de BI podem rodar queries SQL na mesma fonte de dados, sem duplicação.

### **A Camada "Mágica": ***Table Formats*****

### Como você transforma um monte de arquivos soltos num bucket  qualquer (nuvem) em uma tabela confiável com suporte a transações? **Através dos Table Formats** (Formatos de Tabela).

### Existem três tecnologias principais que habilitam o Lakehouse hoje:

1. ### **Delta Lake** (Criado pela **Databricks**, agora *Linux Foundation*).
2. ### **Apache Iceberg** (Criado pela **Netflix**) (**Que será utilizado neste MVP nas camadas Prata e Ouro**).
3. ### **Apache Hudi** (Criado pela Uber).

### Essas tecnologias criam uma camada de log de transações. Quando você faz um UPDATE ou DELETE, o sistema não altera o arquivo original imediatamente; ele registra a mudança no log e gerencia versões dos arquivos, permitindo até mesmo "viajar no tempo" (Time Travel) para ver como os dados estavam ontem.

### **Arquitetura Medalhão** (**Medallion Architecture**)
<center>
<img src="https://drive.google.com/uc?id=1tgFGNhwxYVR6LeTWLzl9pz7mObe8UW6B" alt="Arquitetura Medalhão" width="700">
<center />

### É muito comum organizar os dados dentro de um Lakehouse em três camadas lógicas para garantir a qualidade:

1.	### **Bronze (Raw)**: Dados brutos, exatamente como chegaram da fonte. Nenhuma limpeza é feita aqui.
2.	### **Silver (Refined)**: Dados limpos, filtrados e com tipos corrigidos. É a "verdade única" da organização.
3.	### **Gold (Curated)**: Dados agregados, modelados (ex: Star Schema) e prontos para consumo em dashboards e relatórios executivos.

### Além disso, não vamos utilizar uma stack proprietária, vamos utilizar um conceito bem atual chamado **Modern Data Stack in a Box**, onde mostraremos quais ferramentas vamos usar em nossa solução.

### **O Conceito: "Modern Data Stack in a Box"**
### Refere-se à implantação de um ambiente completo de análise de dados em uma única máquina (como um laptop ou servidor robusto). Essa abordagem utiliza ferramentas open-source eficientes, eliminando a complexidade e o custo de Data Warehouses em nuvem para muitos casos de uso.
### **Componentes Principais:**
### A arquitetura replica as camadas de uma stack moderna tradicional, mas otimizada para execução local:
- ### **Ingestão (EL)**: Ferramentas como *Airflow* ou *Airbyte* para extrair e carregar dados.
- ### **Armazenamento (OLAP)**: Uso de bancos de dados analíticos embutidos de alta performance, sendo o *DuckDB* o principal protagonista.
- ### **Transformação (T)**: Uso do *dbt* para limpeza e modelagem via *SQL* com controle de versão.
- ### **BI & Analytics**: Visualização com *Apache Superset*, *Metabase* ou *Streamlit*.

### **Benefícios**
- ### **Eficiência de Custo**: Elimina gastos recorrentes com infraestrutura de nuvem.
- ### **Produtividade**: Simplifica o desenvolvimento e permite Provas de Conceito (PoCs) rápidas sem configurar sistemas distribuídos.
- ### **Performance**: Tira proveito do hardware moderno local para processamento rápido.


### **Trade-off** (**O contraponto**)
### Essa abordagem oferece controle total e portabilidade. **Entretanto**, a principal concessão é a perda da "escalabilidade infinita" das plataformas de nuvem (como *Snowflake* ou *BigQuery*), já que o sistema está limitado ao hardware da máquina única, o que pode ser um gargalo para volumes massivos de dados ou alta concorrência de usuários. Para o nosso trabalho, a estrutura que temos vai funcionar muito bem, e na seção Conclusão em ***“O que será feito nessa solução para um futuro escalável e incerto?”*** explicaremos como podemos melhorar essa infraestrutura para escala e otimização futura quando os dados crescerem demais.

### **Sobre o uso da Arquitetura neste MVP**
### A arquitetura segue o paradigma de *Data Lakehouse*, unificando a flexibilidade de armazenamento de baixo custo (*Google Cloud Storage*) com a performance de consulta SQL (*Dremio*), sem a necessidade de um Data Warehouse proprietário caro.
### A solução foi desenhada para ser *agnóstica de nuvem* na camada de computação, rodando inteiramente em *contêineres Docker*, o que garante portabilidade e reprodutibilidade."
### A arquitetura utilizada nesse MVP eleva o nível de *"Modern Data Stack"* implementando um padrão avançadíssimo conhecido como **WAP** (*Write-Audit-Publish* ou *Escrever-Auditar-Publicar*), trazendo a semântica do *Git* (*branches*, *commits*, *merges*) para a engenharia de dados.
### O uso do *Project Nessie* como catálogo para gerenciar tabelas *Iceberg* é o grande diferencial aqui. Isso resolve um dos maiores pesadelos da engenharia de dados: a inconsistência de dados durante o processamento.

### Chega de conversa, vamos pra parte prática: nosso MVP funcionando propriamente dito.

<center>
<img src="https://drive.google.com/uc?id=16Tbkf1vEDtZJaxeBMrT7U6XAQv7cykey" alt="Talk is cheap, show me the code" width="500">
<center />

## **Base de Dados alvo**

### Como dissemos anteriormente, não utilizamos apenas uma base de dados, mas multiplas de dentro do site dados.rio através do Google BigQuery. Abaixo, mostramos o catálogo de metadados baseado nas informações do próprio provedor dos dados: A Prefeitura da Cidade do Rio de Janeiro.
### **Disclaimer**: ***Infelizmente o tamanho da fonte nas tabelas ficou pequeno por limitações do próprio Colab.***

### **Catálogo de Dados**



### **Dataset**: *reclamacoes_1746*
### **Tabela**: *chamados*
### **Descrição**: Essa tabela detém todos os dados de reclamações da Central 1746 da Cidade do Rio de Janeiro, mas os dados dessa tabela são apenas sobre transportes de uma maneira geral. Aqui não conseguimos separar por empresa, consórcio ou dados que identifiquem o alvo das reclamações.

| Nome do Campo | Tipo | Modo | Descrição |
| :--- | :--- | :--- | :--- |
| **id_unidade_organizacional** | STRING | NULLABLE | Identificador único, no banco de dados, do órgão que executa o chamado. Por exemplo: identificador da COMLURB quando o chamado é relativo a limpeza urbana. |
| **nome_unidade_organizacional** | STRING | NULLABLE | Nome do órgão que executa a demanda. Por exemplo: COMLURB quando a demanda é relativa a limpeza urbana. |
| **id_unidade_organizacional_mae** | STRING | NULLABLE | ID da unidade organizacional mãe do orgão que executa a demanda. Por exemplo: "CVA - Coordenação de Vigilância de Alimentos" é quem executa a demanda e obede a unidade organizacional mãe "IVISA-RIO - Instituto Municipal de Vigilância Sanitária, de Zoonoses e de Inspeção Agropecuária". A coluna se refere ao ID deste último. |
| **unidade_organizacional_ouvidoria** | STRING | NULLABLE | Booleano indicando se o chamado do cidadão foi feita Ouvidoria ou não. 1 caso sim, 0 caso não. |
| **categoria** | STRING | NULLABLE | Categoria do chamado. Exemplo: Serviço, informação, sugestão, elogio, reclamação, crítica. |
| **id_tipo** | STRING | NULLABLE | Identificador único, no banco de dados, do tipo do chamado. Ex: Iluminação pública. |
| **tipo** | STRING | NULLABLE | Nome do tipo do chamado. Ex: Iluminação pública. |
| **id_subtipo** | STRING | NULLABLE | Identificador único, no banco de dados, do subtipo do chamado. Ex: Reparo de lâmpada apagada. |
| **subtipo** | STRING | NULLABLE | Nome do subtipo do chamado. Ex: Reparo de lâmpada apagada. |
| **status** | STRING | NULLABLE | Status do chamado. Ex. Fechado com solução, aberto em andamento, pendente etc. |
| **longitude** | FLOAT | NULLABLE | Longitude do lugar do evento que motivou o chamado. |
| **latitude** | FLOAT | NULLABLE | Latitude do lugar do evento que motivou o chamado. |
| **data_alvo_finalizacao** | DATETIME | NULLABLE | Data prevista para o atendimento do chamado. Caso prazo_tipo seja D fica em branco até o diagnóstico ser feito. |
| **data_alvo_diagnostico** | DATETIME | NULLABLE | Data prevista para fazer o diagnóstico do serviço. Caso prazo_tipo seja F esta data fica em branco. |
| **data_real_diagnostico** | DATETIME | NULLABLE | Data em que foi feito o diagnóstico do serviço. Caso prazo_tipo seja F esta data fica em branco. |
| **tempo_prazo** | INTEGER | NULLABLE | Prazo para o serviço ser feito. Em dias ou horas após a abertura do chamado. Caso haja diagnóstico o prazo conta após se fazer o diagnóstico. |
| **prazo_unidade** | STRING | NULLABLE | Unidade de tempo utilizada no prazo. Dias ou horas. D ou H. |
| **prazo_tipo** | STRING | NULLABLE | Diagnóstico ou finalização. D ou F. Indica se a chamada precisa de diagnóstico ou não. Alguns serviços precisam de avaliação para serem feitos, neste caso é feito o diagnóstico. Por exemplo, pode de árvore. Há a necessidade de um engenheiro ambiental verificar a necessidade da poda ou não. |
| **dentro_prazo** | STRING | NULLABLE | Indica se a data alvo de finalização do chamado ainda está dentro do prazo estipulado. |
| **situacao** | STRING | NULLABLE | Identifica se o chamado foi encerrado. |
| **tipo_situacao** | STRING | NULLABLE | Indica o status atual do chamado entre as categorias Atendido, Atendido parcialmente, Não atendido, Não constatado e Andamento. |
| **justificativa_status** | STRING | NULLABLE | Justificativa que os órgãos usam ao definir o status. Exemplo: SEM POSSIBILIDADE DE ATENDIMENTO - justificativa: Fora de área de atuação do municipio. |
| **reclamacoes** | INTEGER | NULLABLE | Quantidade de reclamações. |
| **extracted_at** | TIMESTAMP | NULLABLE | Data e hora em que o registro foi extraído pelo Airbyte. |
| **updated_at** | STRING | NULLABLE | Data da ultima atualização. |
| **data_particao** | DATE | NULLABLE | Data de partição dos dados. Trunc(data_inicio). |

### **Dataset**: *clima_pluviometro*
### **Tabela**: *estacoes_alertario*
### **Descrição**: Dados das unidades meteorológicas com seus respectivos dados, serve como dado mestre para a tabela de *taxa_precipitacao_alertario*.

| Nome do Campo | Tipo | Modo | Descrição |
| :--- | :--- | :--- | :--- |
| **id_estacao** | STRING | NULLABLE | Identificador único da estação (meteorológica, pluviométrica ou de monitoramento) no banco de dados. |
| **estacao** | STRING | NULLABLE | Nome ou designação comum da estação. Exemplo: "Copacabana", "Tijuca", "São Cristóvão". |
| **latitude** | FLOAT | NULLABLE | Coordenada geográfica (latitude) da localização da estação em graus decimais. |
| **longitude** | FLOAT | NULLABLE | Coordenada geográfica (longitude) da localização da estação em graus decimais. |
| **cota** | FLOAT | NULLABLE | Altitude da estação em relação ao nível do mar (geralmente expressa em metros). |
| **x** | FLOAT | NULLABLE | Coordenada cartesiana X no sistema de projeção (provavelmente UTM) para georreferenciamento plano. |
| **y** | FLOAT | NULLABLE | Coordenada cartesiana Y no sistema de projeção (provavelmente UTM) para georreferenciamento plano. |
| **endereco** | STRING | NULLABLE | Endereço físico, logradouro ou ponto de referência onde a estação está instalada. |
| **situacao** | STRING | NULLABLE | Status operacional atual da estação. Exemplo: "Operante", "Nao operante". |
| **data_inicio_operacao** | DATETIME | NULLABLE | Data e hora em que a estação começou a operar e coletar dados. |
| **data_fim_operacao** | DATETIME | NULLABLE | Data e hora em que a estação encerrou suas atividades (caso tenha sido desativada). |
| **data_atualizacao** | DATETIME | NULLABLE | Data e hora da última atualização cadastral ou sincronização dos dados da estação. |


### **Dataset**: *clima_pluviometria*

### **Tabela**: *taxa_precipitacao_alertario*
### **Descrição**: Informações sobre o tempo na Cidade do Rio de Janeiro com informações de acumulados de chuva de 15 minutos, 1 hora, 4 horas, 24 horas e 96 horas.

| Nome do Campo | Tipo | Modo | Descrição |
| :--- | :--- | :--- | :--- |
| **primary_key** | STRING | NULLABLE | Chave primária criada a partir da coluna id_estacao e da data_medicao. Serve para evitar dados duplicados. |
| **id_estacao** | STRING | NULLABLE | Identificador único da estação de monitoramento que registrou a medição. (Chave estrangeira para a tabela de estações). |
| **acumulado_chuva_15_min** | FLOAT | NULLABLE | Acumulado de chuva em 15 minutos. |
| **acumulado_chuva_1_h** | FLOAT | NULLABLE | Acumulado de chuva em 1 hora. |
| **acumulado_chuva_4_h** | FLOAT | NULLABLE | Acumulado de chuva em 4 horas. |
| **acumulado_chuva_24_h** | FLOAT | NULLABLE | Acumulado de chuva em 24 horas. |
| **acumulado_chuva_96_h** | FLOAT | NULLABLE | Acumulado de chuva em 96 horas. |
| **horario** | TIME | NULLABLE | Horário no qual foi realizada a medição. |
| **data_particao** | DATE | NULLABLE | Data em que foi realizada a medição. |


### **Dataset**: *transporte_rodoviario_municipal*
### **Tabela**: *licenciamento_frota*
### **Descrição**: Tabela sobre o cadastro da frota de Ônibus da Cidade do Rio de Janeiro. Aqui podemos verificar a idade da frota e podemos cruzar com a tabela *viagem_onibus* onde podemos obter informações sobre o numero de ordem, consórcio, placa e etc.

| Nome do Campo | Tipo | Modo | Descrição |
| :--- | :--- | :--- | :--- |
| **carroceria** | STRING | NULLABLE | Descrição do modelo da carroceria. |
| **id_chassi** | INTEGER | NULLABLE | Código do modelo do chassi. |
| **id_fabricante_chassi** | INTEGER | NULLABLE | Identificador do fabricante do chassi. |
| **nome_chassi** | STRING | NULLABLE | Descrição do modelo do chassi. |
| **id_planta** | INTEGER | NULLABLE | Código da planta do veículo. |
| **tipo_veiculo** | STRING | NULLABLE | Tipo de veículo. |
| **status** | STRING | NULLABLE | Status do veículo. |
| **data_inicio_vinculo** | DATE | NULLABLE | Data de início do vínculo do veículo no STU. |
| **data_ultima_vistoria** | DATE | NULLABLE | Data da última vistoria do veículo. |
| **ano_ultima_vistoria** | INTEGER | NULLABLE | Ano atualizado da última vistoria realizada pelo veículo. |
| **ultima_situacao** | STRING | NULLABLE | Última situação do veículo na data atual. |
| **tecnologia** | STRING | NULLABLE | Tecnologia utilizada no veículo [BASICO, MIDI, MINI, PADRON, ARTICULADO]. |
| **quantidade_lotacao_pe** | INTEGER | NULLABLE | Capacidade de passageiros em pé. |
| **quantidade_lotacao_sentado** | INTEGER | NULLABLE | Capacidade de passageiros sentados. |
| **tipo_combustivel** | STRING | NULLABLE | Tipo de combustível utilizado. |
| **indicador_ar_condicionado** | BOOLEAN | NULLABLE | Indicador se possui ar condicionado [Verdadeiro/Falso]. |
| **indicador_elevador** | BOOLEAN | NULLABLE | Indicador se possui elevador [Verdadeiro/Falso]. |
| **indicador_usb** | BOOLEAN | NULLABLE | Indicador se tem USB [Verdadeiro/Falso]. |
| **indicador_wifi** | BOOLEAN | NULLABLE | Indicador se tem Wi-fi [Verdadeiro/Falso]. |
| **indicador_veiculo_lacrado** | BOOLEAN | NULLABLE | Indicador se o veículo estava lacrado na data atual [Verdadeiro/Falso]. |
| **indicador_data_ultima_vistoria_tratada** | BOOLEAN | NULLABLE | Indica se a data da última vistoria original passou por algum processo de tratamento, limpeza ou normalização. |
| **data_arquivo_fonte** | DATE | NULLABLE | Data do arquivo do STU com os dados utilizados. |
| **versao** | STRING | NULLABLE | Código de controle de versão [SHA do GitHub]. |
| **datetime_ultima_atualizacao** | DATETIME | NULLABLE | Última atualização [GMT-3]. |
| **id_execucao_dbt** | STRING | NULLABLE | Identificador da execução do DBT que modificou o dado pela última vez. |
| **ano_ultima_vistoria_atualizado** | INTEGER | NULLABLE | Ano correspondente à última vistoria, recalculado ou corrigido após processos de tratamento de dados. |


### **Dataset**: *transporte_rodoviario_municipal*
### **Tabela**: *viagem_onibus*
### **Descrição**: Essa é a tabela na qual temos as informações das viagens de todos os ônibus que circulam na Cidade do Rio de Janeiro.

| Nome do Campo | Tipo | Modo | Descrição |
| :--- | :--- | :--- | :--- |
| **data** | DATE | NULLABLE | Data da viagem. |
| **consorcio** | STRING | NULLABLE | Consórcio ao qual o serviço pertence. |
| **tipo_dia** | STRING | NULLABLE | Dia da semana - categorias: Dia Útil, Sábado, Domingo e Ponto Facultativo. |
| **id_empresa** | STRING | NULLABLE | Código identificador da empresa que opera o veículo. |
| **id_veiculo** | STRING | NULLABLE | Código identificador do veículo [número de ordem]. |
| **id_viagem** | STRING | NULLABLE | Código único identificador da viagem. |
| **servico** | STRING | NULLABLE | Serviço realizado pelo veículo [com base na identificação do trajeto]. |
| **shape_id** | STRING | NULLABLE | Código identificador do shape [trajeto]. |
| **sentido** | STRING | NULLABLE | Sentido da linha. |
| **datetime_partida** | DATETIME | NULLABLE | Horário de início da viagem. |
| **datetime_chegada** | DATETIME | NULLABLE | Horário de fim da viagem. |
| **tempo_viagem** | INTEGER | NULLABLE | Tempo aferido da viagem (minutos). |
| **distancia_planejada** | FLOAT | NULLABLE | Distância do shape [trajeto] planejado (km). |
| **perc_conformidade_shape** | FLOAT | NULLABLE | Percentual de sinais emitidos dentro do shape [trajeto] ao longo da viagem. |
| **perc_conformidade_registros** | FLOAT | NULLABLE | Percentual de minutos da viagem com registro de sinal de GPS. |
| **versao_modelo** | STRING | NULLABLE | Código de controle de versão [SHA do GitHub]. |

### Essas são as tabelas que serão convertidas em outros conjuntos de dados para gerar os insights e responder as questões presentes no *objetivo* desse trabalho.

## **Coleta e Ingestão de Dados para a Camada Bronze**

### **Origem dos Dados e Desafios Iniciais**
Devido a restrições de acesso via Conta de Serviço ao Google BigQuery do **data.rio**, a estratégia de coleta foi adaptada. Realizamos a extração manual das tabelas para arquivos **.csv** (limitados a 1 GB cada) e o upload para uma estrutura organizada no **Google Drive**.

![Estrutura de Pastas com arquivos coletados](https://drive.google.com/uc?id=13RVLAW-ASzXjo-M8gAUSJbcO7OZtEr7W)



### O volume total coletado é de **6,71 GB**. Embora modesto para padrões de Big Data, este volume é ideal para demonstrar a eficiência de ferramentas *open source* em processar dados que superam a capacidade de memória convencional de máquinas locais, utilizando técnicas de *Lazy Loading* e *Streaming*.

![Estrutura de dentro de uma pasta](https://drive.google.com/uc?id=1irSDz-fItos9fnblISWjphs-3LyxAOpd)

---

### **Orquestração com Apache Airflow**
### Para automatizar a movimentação desses dados para a nuvem, utilizamos o **Apache Airflow** rodando em uma VPS via Docker. O Airflow é uma plataforma de código aberto para agendar e monitorar fluxos de trabalho.

<img src="https://drive.google.com/uc?id=1cRHfKNydWC-uFUzitbJgXqJL-zTyML5O" alt="Arquitetura do Airflow" width="700">


* ### **O que é uma DAG?** No Airflow, o pipeline é definido como uma **DAG (Directed Acyclic Graph)**. Trata-se de um conjunto de tarefas organizadas de forma que reflitam suas dependências e relações, garantindo que o fluxo de dados siga uma direção lógica sem loops infinitos.
* ### **Pipeline de Ingestão (Factory Pattern):** Nossa DAG foi desenvolvida utilizando o padrão *Factory* (Orchestrator). Isso permite que novos datasets sejam adicionados apenas via configuração, garantindo que a orquestração cresça sem a necessidade de reescrever o código principal.

### **Componentes do Diretório `dags/`**

### A lógica do pipeline está segmentada para garantir modularidade, seguindo princípios de arquitetura de software:

* ### **Entrypoint**: É o [arquivo](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/mvp_ingestion_pipeline.py) de entrada que instancia o `PipelineOrchestrator` e expõe o objeto `dag` para o Airflow.
* ### [**Classe** `PipelineOrchestrator`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/mvp_orchestrator_dag.py) : Responsável por iterar sobre os datasets e criar dinamicamente as tarefas e dependências. Aqui implementamos a ordem crítica de execução: **Branch -> Ingestão -> Registro -> Merge**.
* ### [**Classe** `IngestionManager`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/ingestion_manager.py): O motor de processamento. Utiliza o **DuckDB** para converter CSV em Parquet e realiza validações de vacuidade antes do upload.
* ### [**Classe** `DataOpsManager`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/dataops_manager.py): O componente de governança. Gerencia o **Project Nessie** via ODBC, criando branches isoladas para o ETL e garantindo o registro de tabelas Iceberg no Dremio.

### Essa separação garante que alterações na lógica de transformação (DuckDB) ou na governança (Nessie) não quebrem a estrutura da orquestração.

---

### **Fluxo de Dados e Padrão WAP (Write-Audit-Publish)**
### O processo segue o padrão rigoroso de **WAP** para garantir que a ramificação principal (*main*) nunca receba dados corrompidos:

1.  ### **Isolamento (Write):** O `DataOpsManager` cria uma branch temporária no Nessie (ex: `dev_20251220`). Toda a escrita ocorre neste ambiente isolado.
2.  ### **Ingestão e Conversão:** O DuckDB lê os CSVs em *streaming* e gera arquivos **Parquet** com tipagem automática.
3.  ### **Auditoria Bronze (Audit):** Antes do upload, validamos se o arquivo não está vazio. Na camada Bronze, o dado é mantido em formato *Raw* para preservar a linhagem completa.
4.  ### **Registro e Publicação (Publish):** Registramos a tabela no Dremio apontando para o GCS simplificado (`dataset/ano=YYYY/mes=MM/dia=DD`). Após o sucesso, realizamos o **Merge** atômico para a branch `main`.

---

### **Segurança e Governança (IAM)**
### A segurança é baseada no princípio do **Privilégio Mínimo** via **IAM**:

* ### **Service Account (SA):** Identidade exclusiva para o Airflow com chaves JSON criptografadas.
* ### **Escopo:** Leitura restrita no Drive e permissão `Storage Object Admin` nos buckets de Landing e Bronze.

> ### **Nota de Arquiteto:** Nenhuma credencial é exposta. Configurações de IDs e Buckets são injetadas via variáveis de ambiente protegidas no container.

---

### **Repositório de Código**
### Toda a lógica de orquestração e as classes de gerenciamento estão versionadas no GitHub:

> 🔗 [**Acessar Diretório das DAGs no GitHub**](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/tree/master/dags)

---

### **Detalhes Técnicos: Implementando o Bypass**
### Enfrentamos um grande desafio na comunicação do Airflow com o Google Drive. Como mostra o vídeo abaixo:

<center>
<img src="https://drive.google.com/uc?id=1Mow0LfPljsZs_S4YuHADUR18bS7RGZxE" alt="Talk is cheap, show me the code" width="500">
<center />

### Para resolver, implementamos um *bypass* direto via Google Discovery API no `IngestionManager`, substituindo o Hook padrão por uma solução mais estável:

### **Antes (Padrão Airflow):**
```python
from airflow.providers.google.cloud.hooks.drive import GoogleDriveHook

def __init__(self):
    self._drive_hook = GoogleDriveHook(gcp_conn_id="google_cloud_default")

def list_files(self, folder_id):
    return self._drive_hook.get_conn().files().list(q=f"'{folder_id}' in parents").execute()
```

### **Depois (Bypass Direto - IngestionManager):**
```python
from google.oauth2 import service_account
from googleapiclient.discovery import build

def __init__(self):
    self._key_path = "/opt/airflow/keys/sa-airflow.json"
    self._scopes = ['[https://www.googleapis.com/auth/drive.readonly](https://www.googleapis.com/auth/drive.readonly)']

def _get_drive_service(self):
    creds = service_account.Credentials.from_service_account_file(
        self._key_path, scopes=self._scopes
    )
    return build('drive', 'v3', credentials=creds)
```
### **Integração com Dremio (DataOpsManager)**
### Para finalizar a camada Bronze, automatizamos o registro das tabelas como Apache Iceberg, permitindo que o Dremio gerencie versões de forma atômica:
```python
def create_dag(self, dag_id: str, schedule_interval):
        with DAG(
            dag_id=dag_id,
            default_args=self._default_args,
            description="Pipeline de Ingestão: Drive -> Landing -> Bronze (Parquet)",
            schedule_interval=schedule_interval,
            catchup=False,
            tags=["ingestao", "bronze", "duckdb", "poo"],
        ) as dag:

            # 1. Loop de Datasets (Configurado sem as reclamações)
            for dataset_name, folder_id in self._datasets_config.items():

                if not folder_id:
                    continue

                # 2. Branch ÚNICA por Dataset (Ex: dev_viagens_onibus_20251220)
                # Isso isola o trabalho e evita que um erro "apague" o progresso de outros
                branch_name = f"dev_{dataset_name}_{{{{ ds_nodash }}}}"

                # 2.1 Criar a branch para este dataset
                create_branch_task = PythonOperator(
                    task_id=f"create_branch_{dataset_name}",
                    python_callable=self._dataops.create_branch,
                    op_kwargs={"branch_name": branch_name},
                )

                # 2.2 Ingestão (Drive -> GCS)
                ingest_task = PythonOperator(
                    task_id=f"ingest_{dataset_name}",
                    python_callable=self._ingestor.process_file_to_bronze,
                    op_kwargs={
                        "folder_id": folder_id,
                        "bucket_landing": self._bucket_landing,
                        "bucket_bronze": self._bucket_bronze,
                        "dataset_name": dataset_name,
                        "data_referencia": "{{ ds }}",
                    },
                )

                # 2.3 Registro (Dremio/Nessie) com Tipagem Forte
                register_task = PythonOperator(
                    task_id=f"register_{dataset_name}_on_dremio",
                    python_callable=self._dataops.ensure_table_exists,
                    op_kwargs={
                        "dataset_name": dataset_name,
                        "branch_name": branch_name,
                        "bucket_name": self._bucket_bronze,
                        "schema_string": SCHEMAS.get(dataset_name),
                    },
                )

                # 2.4 Merge Individual (Publica apenas este dataset na Main)
                merge_task = PythonOperator(
                    task_id=f"merge_{dataset_name}_to_main",
                    python_callable=self._dataops.merge_branch,
                    op_kwargs={"branch_name": branch_name, "target_ref": "main"},
                )

                # Define o fluxo isolado: Branch -> Ingest -> Register -> Merge
                create_branch_task >> ingest_task >> register_task >> merge_task

            return dag
```
```python
class DataOpsManager:
    def __init__(self, conn_id="dremio_odbc", catalog="nessie_catalog") -> None:
        self._conn_id = conn_id
        self._catalog = catalog
        self._hook = OdbcHook(odbc_conn_id=conn_id)

    def _execute_sql_direct(self, sql: str):
        """
        Bypass Spartan: Força o autocommit=True em todas as execuções.
        Impede que o Airflow tente desabilitar o autocommit (causador do HYC00).
        """
        conn_str = self._hook.odbc_connection_string
        logging.info(f"Dremio SQL Exec: {sql}")

        with pyodbc.connect(conn_str, autocommit=True) as conn:
            with conn.cursor() as cursor:
                cursor.execute(sql)

    def create_branch(self, branch_name: str, source_ref="main"):
        sql = f'CREATE BRANCH "{branch_name}" IN {self._catalog}'
        try:
            self._execute_sql_direct(sql)
            logging.info(f"Branch '{branch_name}' pronta.")
        except Exception as e:
            if "already exists" in str(e).lower():
                logging.warning(f"Branch '{branch_name}' já existe.")
            else:
                raise e

    def ensure_table_exists(
        self,
        dataset_name: str,
        branch_name: str,
        bucket_name: str,
        schema_string: str = None,
    ):
        """
        Cria a tabela no Nessie usando CAST para garantir os tipos do catálogo.
        """
        use_ref = f'USE REFERENCE "{branch_name}" IN {self._catalog}'

        if schema_string:
            # Transforma "col TIPO" em "CAST(col AS TIPO) as col"
            cols_with_types = [c.strip() for c in schema_string.split(",")]
            cast_list = []
            for item in cols_with_types:
                parts = item.split()
                if len(parts) >= 2:
                    col_name = parts[0]
                    col_type = parts[1]
                    cast_list.append(f"CAST({col_name} AS {col_type}) as {col_name}")

            select_clause = ", ".join(cast_list)
        else:
            select_clause = "*"

        # Criamos via CTAS. O Dremio gerencia o LOCATION automaticamente no Nessie.
        sql_create = f"""
            CREATE TABLE IF NOT EXISTS {self._catalog}.{dataset_name}
            AS SELECT {select_clause}
            FROM TABLE(gcs_bronze."{bucket_name}"."{dataset_name}" (type => 'parquet'))
        """

        try:
            logging.info(f"Registrando {dataset_name} com tipagem explícita...")
            self._execute_sql_direct(use_ref)
            self._execute_sql_direct(sql_create)
            logging.info(f"Tabela {dataset_name} criada com sucesso.")
        except Exception as e:
            if "already exists" in str(e).lower():
                logging.warning(f"Tabela {dataset_name} já existe na branch.")
            else:
                raise e

    def merge_branch(self, branch_name: str, target_ref="main"):
        """
        Publica os dados da branch de dev para a main.
        """
        sql = f'MERGE BRANCH "{branch_name}" INTO "{target_ref}" IN {self._catalog}'
        try:
            logging.info(f"Publicando dados: {branch_name} -> {target_ref}")
            self._execute_sql_direct(sql)
            logging.info("Merge concluído!")
        except Exception as e:
            raise e
```

### **Evolução da Estratégia de Ingestão: Do Funil ao Isolamento**
### Inicialmente, o pipeline foi desenhado em um modelo de "funil", onde todos os datasets (viagens, clima, frota e reclamações) convergiam para uma única branch de desenvolvimento antes do merge final para a produção (`main`).

* ### **O Problema (Gargalo)**: Como demonstrado na Figura 1, uma falha em um único dataset (neste caso, `reclamacoes_1746`) causou o erro `upstream_failed` na tarefa de publicação final. Isso impedia que mesmo os dados saudáveis chegassem aos usuários finais, caracterizando um ponto único de falha.

* ### **A Solução (Isolamento de Branches)**: A arquitetura foi refatorada para um modelo de "trilhos paralelos" (Video abaixo). Cada dataset passou a ter seu próprio ciclo de vida completo: criação de branch dedicada, ingestão, registro e merge individual.
### **Foram 10 horas de um dia para resolver todos os problemas de integração entre o Airflow, Dremio e Nessie.**


<center>
<img src="https://drive.google.com/uc?id=1eHWavX9H0tbxpGu1qYbnrhPxT2phiwOt" alt="Pipeline do Airflow executado com sucesso!">
<center />

*Arquitetura final do pipeline de dados no Apache Airflow, demonstrando o paralelismo total e o isolamento transacional via branches do Project Nessie para cada entidade do domínio de transportes e clima.*

### **Ciclo de Vida do Dado no Lakehouse**
### A estrutura visível da gravação acima detalha as quatro etapas críticas para garantir a integridade e a disponibilidade de cada tabela. Vamos passear por cada uma delas com o devido código fonte:


### **1. Criação de Branch Efêmera (`create_branch`)**: Para cada execução, o Project Nessie cria uma ramificação isolada (ex: `dev_clima_pluviometria_20251220`). Isso permite que operações de escrita ocorram sem risco de corrupção da branch principal.
---

```python
def create_branch(self, branch_name: str, source_ref="main"):
        sql = f'CREATE BRANCH "{branch_name}" IN {self._catalog}'
        try:
            self._execute_sql_direct(sql)
            logging.info(f"Branch '{branch_name}' pronta.")
        except Exception as e:
            if "already exists" in str(e).lower():
                logging.warning(f"Branch '{branch_name}' já existe.")
            else:
                raise e
```
*Trecho de código pertencente a [classe `DataOpsManager`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/dataops_manager.py)*


```python
create_branch_task = PythonOperator(
                    task_id=f"create_branch_{dataset_name}",
                    python_callable=self._dataops.create_branch,
                    op_kwargs={"branch_name": branch_name},
                )
```
*Trecho de código pertence a [classe `PipelineOrchestrator`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/mvp_orchestrator_dag.py)*

![GitDataBranch](https://drive.google.com/uc?id=1uymNNpOhkOJHlcP1WZ_iJxok8a2fPRNm)

*Interface de gerenciamento do Project Nessie exibindo o histórico cronológico de commits. Cada entrada representa uma operação atômica de DDL (Data Definition Language) ou DML (Data Manipulation Language), assegurando a auditoria completa e a capacidade de reprodutibilidade temporal (Time Travel) sobre o catálogo Iceberg.*

### **Diagrama de Linhagem: Modelo Git Flow para Dados**

### Podemos representar o fluxo de trabalho de dados para a camada Bronze com o seguinte grafo, que ilustra o paralelismo das branches individuais:

* ### **Branches Efêmeras por Dataset**: Diferente de um fluxo Git tradicional onde as branches duram dias, no seu Lakehouse as branches são efêmeras e criadas programaticamente pelo Airflow para cada tarefa de ingestão. Isso garante que a branch `main` nunca seja exposta a estados intermediários ou falhas de schema.

* ### **Commits de DDL (Data Definition Language)**: Cada "bolinha" no grafo (commit) representa a materialização física dos metadados Iceberg. No seu caso, esses commits carregam a **tipagem forte** (CAST) que você definiu no catálogo, garantindo que o dado na `main` seja imutável e confiável.

* ### **Merge Atômico**: As linhas que voltam para a `main` representam o momento em que o dado se torna visível para o Dremio. Como demonstrado na Figura anterior, o Nessie registra o hash de cada merge, permitindo que você faça o "Time Travel" (viagem no tempo) para qualquer ponto desse gráfico.

![Git-Graph dos dados](https://drive.google.com/uc?id=1U72uFHAGYk_hnNprAXydVCzClD_Qckr3)

*Grafo de versionamento do catálogo Nessie. A estrutura demonstra o isolamento transacional onde cada dataset (Clima, Viagens, Frota, Estações) opera em uma ramificação independente, culminando em merges atômicos na branch principal (main) após a validação do esquema.*

### **2. Ingestão via Edge ETL (`ingest_task`)**: Processamento local utilizando DuckDB para transformar arquivos brutos em Parquet otimizado no Google Cloud Storage (GCS).

```python
ingest_task = PythonOperator(
                    task_id=f"ingest_{dataset_name}",
                    python_callable=self._ingestor.process_file_to_bronze,
                    op_kwargs={
                        "folder_id": folder_id,
                        "bucket_landing": self._bucket_landing,
                        "bucket_bronze": self._bucket_bronze,
                        "dataset_name": dataset_name,
                        "data_referencia": "{{ ds }}",
                    },
                )
```
*Trecho de código pertence a [classe `PipelineOrchestrator`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/mvp_orchestrator_dag.py)*

### **3. Registro com Schema Enforcement (`register_task`)**: A tabela é materializada no catálogo através de comandos `CREATE TABLE AS SELECT` com tipagem forte (`CAST`). Isso resolve o problema de inconsistência de tipos observado anteriormente.

```python
register_task = PythonOperator(
                    task_id=f"register_{dataset_name}_on_dremio",
                    python_callable=self._dataops.ensure_table_exists,
                    op_kwargs={
                        "dataset_name": dataset_name,
                        "branch_name": branch_name,
                        "bucket_name": self._bucket_bronze,
                        "schema_string": SCHEMAS.get(dataset_name),
                    },
                )
```
*Trecho de código pertence a [classe `PipelineOrchestrator`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/mvp_orchestrator_dag.py)*

### **4. Merge Atômico (`merge_task`)**: Assim que o registro é validado, a branch é mesclada na `main`, tornando o dado disponível para consumo no Dremio e PowerBI de forma instantânea e segura.

```python
merge_task = PythonOperator(
                    task_id=f"merge_{dataset_name}_to_main",
                    python_callable=self._dataops.merge_branch,
                    op_kwargs={"branch_name": branch_name, "target_ref": "main"},
                )
```
*Trecho de código pertence a [classe `PipelineOrchestrator`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/mvp_orchestrator_dag.py)*

## **Benefícios Técnicos Implementados**

* ### **Versionamento Nativo (Git-for-Data)**: Através do Nessie, o histórico de cada alteração é registrado como um commit, permitindo auditoria total e a possibilidade de realizar *rollbacks* se necessário.

* ### **Disponibilidade Parcial**: Mesmo que um dataset apresente erros, as outras esteiras continuam operacionais, garantindo que o Lakehouse seja atualizado com os dados disponíveis.

* ### **Consistência ACID**: O uso do formato Apache Iceberg garante que as operações de escrita e leitura sejam consistentes, evitando que usuários vejam dados parciais durante o processo de ingestão.

### E isso foi decisivo para que os dados fossem entregues corretamente no Dremio, como mostra a figura abaixo:

<center>
  <img src="https://drive.google.com/uc?id=1HbXyNpOWtWo8fqiGuGanUU7onz_EHW29" alt="Tabelas criadas no Dremio">
<center />

**Problemas estruturais após a criação da Camada Bronze**

### Tivemos grandes problemas com o ambiente e com isso, tivemos de eliminar o dbt e o Nessie do MVP, motivos dos quais só vamos verificar após a entrega, vide a seção de Autoavaliação para informações adicionais sobre o "Apocalipse" que ocorreu na véspera da entrega.
### Embora o pipeline de ingestão tenha sido validado com sucesso via Airflow até a persistência no Google Cloud Storage, a orquestração do catálogo de metadados foi realizada manualmente no motor Dremio para este MVP. Esta decisão técnica visou garantir a integridade dos tipos de dados (Data Quality) antes da transição para a camada Prata. Em arquiteturas futuras, propõe-se a integração de catálogos automatizados (como Project Nessie) operando em bancos de dados transacionais persistentes (PostgreSQL) para suportar o versionamento de dados em escala.
### Criamos queries que seguem o padrão **CTAS** (*Create Table as Select*) onde vamos executá-las diretamente sobre o Dremio com o Apache Iceberg.
### Tivemos de recriar toda a estrutura da camada Bronze novamente, mas dentro do dremio, simulando o que fazia o Nessie por questões de compatibilidade e fidelidade ao MVP.
### Para tal, criamos uma fonte de dados no Dremio apontando para o bucket da camada Bronze, como mostra a figura abaixo:

![gcs_bronze](https://drive.google.com/uc?id=1_rA6wiKXvH5fbod4Z8r_Ih3PhQXBgxYJ)

### Com a opção CTAS selecionada para *ICEBERG*
![CTAS Habilitado](https://drive.google.com/uc?id=1EsJpcQG75u3qwgqJbRagNBL6eO5WeZC2)

### Tivemos de criar uma nova estrutura em nosso projeto para receber esses scripts .sql:

```plaintext
~/mvp_transporte/
├── sql/                   # Scripts SQL organizados por camada
│   ├── bronze/            # Queries CTAS (as que passei acima)
│   ├── silver/            # Queries de limpeza e enriquecimento
│   └── gold/              # Queries de agregação para dashboards
└── docs/                  # Documentação do projeto
    └── catalog/           # Dicionário de dados e definições de tabelas
```
### **Queries CTAS: Criando a Camada Bronze (Iceberg)**

Diferente da leitura direta do Parquet, o CTAS cria uma tabela gerenciada com metadados Iceberg, o que permite versionamento e performance superiores para a camada Prata. Utilizaremos os esquemas definidos anteriormente para garantir a tipagem forte.

### Criamos uma para cada tabela baseado no catálogo de dados do provedor dos dados, que, inclusive, já mostramos aqui em seções anteriores.

### Escolhemos o namespace `mvp_transporte.bronze` ao invés de `nessie_catalog` como dantes.

### Para cada query será criado um arquivo com extensão .sql para documentação.

### [**Tabela**: `mvp_transporte.bronze.viagens_onibus_iceberg`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/sql/bronze/viagens_onibus.sql)
### [**Tabela** : `mvp_transporte.bronze.clima_pluviometria_iceberg`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/sql/bronze/clima_pluviometria.sql)
### [**Tabela**: `mvp_transporte.bronze.estacoes_clima_iceberg`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/sql/bronze/estacoes_clima.sql)

### **Arquitetura de Catálogo e Persistência no Dremio**
### 1. **O Space: A Camada de Organização Virtual**

### Um **Space** no Dremio (ex: `mvp_transportes`) é uma abstração lógica, funcionando como um Namespace. Entretanto, um Space não possui armazenamento físico próprio; ele não tem um disco rígido ou um bucket atrelado a ele.

* ### Por que criamos? Para organizar o projeto de forma hierárquica (Bronze, Silver, Gold), permitindo aplicar governança e facilitar a descoberta de dados sem expor a complexidade dos caminhos físicos do Google Cloud Storage.

### **A Tabela Física no Bucket: Persistência e Performance**
### Diferente dos Spaces, os **Sources** ou **Fontes de dados** (como a *gcs_bronze*) são as conexões com o mundo real.

* ### **Por que criar a tabela no Bucket?** Como o Dremio não possui armazenamento, toda operação de escrita (`CREATE TABLE`) precisa de um destino físico que suporte arquivos.

* ### Ao apontar para um storage qualquer, no nosso caso o bucket no GCS, o Dremio utiliza o formato **Iceberg** para gravar não apenas os dados, mas também os metadados (manifestos e versões) que garantem a integridade da Camada Bronze.

* ### Isso resolve o erro muito comum na criação de tabelas no Dremio de "*container not found*", pois agora o Dremio sabe exatamente em qual bucket/armazenamento físico deve depositar os arquivos.

### Agora que temos as consultas, vamos executá-las e recomeçar.

### **A View: O Contrato de Dados (Decoupling)**
### A criação de **Views** dentro dos Spaces é a "chave de ouro" da arquitetura.
* ### **Por que usar Views?** Uma View funciona como um **Contrato**. Ela aponta para a tabela física complexa no GCS, mas oferece ao usuário um nome simples e limpo dentro do Space (ex: `mvp_rio_transportes.bronze.viagens_onibus`).

* ### **Vantagem Técnica**: Se no futuro você precisar trocar o bucket do Google Cloud ou migrar para a AWS, você altera apenas a definição da View. Para o analista ou para o dashboard, o nome da tabela permanece inalterado, garantindo **independência de infraestrutura**.

### A arquitetura implementada adotou o conceito de Camada Semântica Desacoplada. Enquanto a persistência física foi consolidada em tabelas gerenciadas no formato **Apache Iceberg** diretamente no Google Cloud Storage (garantindo durabilidade e performance), a interface de consumo foi organizada através de **Virtual Spaces** no Dremio. Esta abordagem permite que a estrutura do projeto (Bronze, Silver e Gold) funcione como um catálogo lógico independente do armazenamento físico, facilitando a manutenção e a evolução do esquema de dados sem impactar as camadas superiores de consumo.

![Tabelas na Camada Bronze](https://drive.google.com/uc?id=1aOWkwqhUBlXOtP_IOZ4Vb340QMj7agKK)

*Tabelas na Camada Bronze dentro do Dremio*


![Views dentro de um Space com um diretório chamado Bronze](https://drive.google.com/uc?id=1guyfm0gLUY5aFd1aYivayZaqR48mfvOm)

*Views dentro de um Space com um diretório chamado Bronze*

### Como podemos notar, existe uma diferença gráfica entre Tabelas e Views e isso facilita bastante o manuseio do ambiente e dos dados.

## **Camada Prata: tratamento dos dados, enriquecimento, criação de Esquema inicial e Carga**

### A construção da camada Prata teve como objetivo central a criação de uma "Single Source of Truth" (SSOT) integrada, preparada para responder às pergunta presentes na seção **Objetivo** deste MVP.

### **O Caminho até a [`stg_viagens`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/sql/silver/stg_viagens.sql): Do Dado Bruto ao Dataset Curado**
### A criação desta tabela foi o resultado de um processo de refinamento iterativo. Partimos de dados brutos na camada Bronze (telemetria e meteorologia) que estavam completamente desconectados em termos de grão e tempo.

### **1. Por que criamos esta tabela?**

* ### **Integração de Domínios Distintos:** Ônibus geram eventos a cada segundo; o clima gera medições a cada 15 minutos. Criamos a [`stg_viagens`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/sql/silver/stg_viagens.sql) para ser o "ponto de encontro", onde cada viagem é carimbada com o estado do clima no momento exato da partida.

* ### **Materialização de Lógica (Feature Engineering)**: Como você preza por **Clean Code**, movemos as regras de negócio complexas (como a classificação de intensidade de chuva e a detecção de fins de semana) da visualização para o processamento. Isso garante que a regra seja escrita uma única vez (DRY - *Don't Repeat Yourself*).

* ### **Padronização de Esquemas**: Passamos de arquivos Parquet sem tipagem rígida para uma tabela **Iceberg**, garantindo integridade de tipos e facilitando a evolução do esquema no futuro.

### **Por que a Wide Table é a melhor escolha para a Camada Prata?**

### Em arquiteturas de Data Lakehouse, a camada Prata atua como a **Staging Area** para a análise. A conclusão de que uma **Wide Table (Tabela Larga)** é o melhor modelo baseia-se em três pilares:

### **A. Eficiência Computacional (O(n) vs. O(n²))**
### A tentativa inicial de manter tabelas separadas e juntá-las na hora da consulta gerou um **produto cartesiano** que travou o sistema por mais de 40 minutos.

![Query levou muito tempo](https://drive.google.com/uc?id=1TcsfQ6XNY5tBMeBEHaGa7-DtUcK4FhDN)

* ### **O Motivo**: Joins em tempo de execução sobre chaves não únicas e calculadas (como horas extraídas de datas) são extremamente custosos.

```sql
CREATE TABLE "gcs_silver".stg_viagens AS
SELECT
    v."data" as data_viagem,
    v.id_viagem,
    v.servico,
    v.consorcio,
    v.tipo_dia,
    -- Campos para responder Perguntas 1 e 2 (Picos e Sazonalidade)
    EXTRACT(HOUR FROM v.datetime_partida) as hora_partida,
    CASE
        WHEN EXTRACT(DOW FROM v."data") IN (0, 6) THEN TRUE
        ELSE FALSE
    END as is_weekend,
    -- Campos para responder Perguntas 3, 4 e 5 (Duração e Consistência)
    v.tempo_viagem,
    v.distancia_planejada,
    -- Campos para responder Perguntas 5 e 6 (Impacto da Chuva)
    COALESCE(c.chuva_1h_no_inicio, 0) as chuva_no_horario,
    COALESCE(c.tipo_chuva, 'Sem Chuva') as condicao_clima
FROM mvp_rio_transportes.bronze.viagens_onibus v
LEFT JOIN mvp_rio_transportes.silver.viagens_com_clima c
    ON v."data" = c.data_particao
    AND EXTRACT(HOUR FROM v.datetime_partida) = EXTRACT(HOUR FROM c.datetime_partida)
```

* ### **A Vantagem da Wide Table**: Ao denormalizar os dados agora, realizamos o Join pesado apenas uma vez. Para a camada Ouro, o custo de consulta será linear, pois todos os atributos necessários já estão na mesma linha.

```sql
CREATE TABLE "gcs_silver".stg_viagens AS
WITH clima_agregado AS (
    SELECT
        data_particao,
        EXTRACT(HOUR FROM horario) as hora_medicao,
        AVG(acumulado_chuva_1_h) as avg_chuva_1h
    FROM mvp_rio_transportes.bronze.clima_pluviometria
    GROUP BY data_particao, EXTRACT(HOUR FROM horario)
)
SELECT
    v."data" as data_viagem,
    v.id_viagem,
    v.servico as linha,
    v.consorcio,
    v.tipo_dia,
    v.distancia_planejada,
    v.tempo_viagem,
    EXTRACT(HOUR FROM v.datetime_partida) as hora_partida,
    CASE WHEN EXTRACT(DOW FROM v."data") IN (1, 7) THEN TRUE ELSE FALSE END as is_weekend,
    COALESCE(c.avg_chuva_1h, 0) as chuva_no_horario,
    CASE
        WHEN c.avg_chuva_1h BETWEEN 0.1 AND 5 THEN 'Fraca'
        WHEN c.avg_chuva_1h BETWEEN 5 AND 25 THEN 'Moderada'
        WHEN c.avg_chuva_1h BETWEEN 25.1 AND 50 THEN 'Forte'
        WHEN c.avg_chuva_1h > 50 THEN 'Muito Forte'
        ELSE 'Sem Chuva'
    END as condicao_clima
FROM mvp_rio_transportes.bronze.viagens_onibus v
LEFT JOIN clima_agregado c
    ON v."data" = c.data_particao
    AND EXTRACT(HOUR FROM v.datetime_partida) = c.hora_medicao;
```
![Criação refatorada levou menos tempo](https://drive.google.com/uc?id=1lLBHoxAw1zSmz_Q0sKw7K05Bu0WnTLaz)


### **B. O Conceito de "Vetor de Resposta"**
* ### **Independência de Perguntas**: Uma Wide Table com 11 colunas bem selecionadas consegue responder a **todas as 6 perguntas de pesquisa** sem precisar voltar à Bronze. Se uma nova pergunta surgir, o "vetor" de dados já possui o contexto climático e temporal pronto para ser filtrado.

### **C. Preservação da Granularidade para Estatística**
### Entretanto, o ponto fundamental é que, embora a tabela seja "larga" em colunas, ela permanece "fina" em linhas (mantém o grão de uma linha por viagem).
* ### Isso é vital para a **Pergunta 4** (Desvio Padrão). Se tivéssemos reduzido os dados para médias prematuramente, perderíamos a capacidade de enxergar a variância e os outliers que ocorrem em dias de chuva forte.


## **Camada Ouro: modelagem de Dimensões e Fatos e Carga para Análise**

## **Modelagem**

## **Carga**

## **Análise**

### **Qualidade dos dados**

### **Solução do Problema**

### **Como essa arquitetura poderia evoluir no futuro?**

## **Autoavaliação**

### **Desafios enfrentados (Post-Mortem)**


### **Aprendizado e o que funcionou?**

### **O Cemitério das boas idéias**

### **O Gargalo da Dependência Monolítica**
### No início do projeto, a orquestração foi desenhada seguindo um modelo linear de dependências, onde múltiplos fluxos de dados convergiam para um único ponto de integração.

* ### **A Dificuldade**: Uma falha em um único dataset (inconsistência de esquema nas reclamações 1746) gerava um efeito cascata que impedia o merge final de todos os outros dados saudáveis.

* ### **A Solução**: Transição para uma arquitetura de Isolamento Transacional, onde cada dataset opera em sua própria branch do Nessie e possui sua própria tarefa de merge independente.

### ** Incompatibilidade de Protocolos: Apache Flight SQL vs. ODBC**
### A integração entre o Airflow (via `PythonOperator`) e o Dremio revelou desafios na camada de transporte de dados.

* ### **A Dificuldade**: O uso do driver ODBC padrão com o protocolo **Flight SQL** do Dremio apresentou problemas de gestão de transações. O Airflow, por padrão, tentava gerenciar o commit, o que entrava em conflito com o motor de execução do Dremio.

* ### **A Solução**: Implementação de um bypass técnico no `DataOpsManager`, forçando a execução direta via `pyodbc` com o parâmetro `autocommit=True`. Isso permitiu que comandos DDL (como o merge de branches) fossem executados de forma atômica e sem interferência do orquestrador.

* ### A **Solução**: Adoção do padrão **CTAS (Create Table As Select)**. O Dremio passou a gerenciar automaticamente o posicionamento dos metadados Iceberg no bucket do GCS, eliminando a complexidade manual de gestão de arquivos.

### **Schema Drift e a Falha da Inferência Automática**
### A confiança na inferência automática de tipos do Dremio mostrou-se um ponto de vulnerabilidade para dados reais e "sujos".

* ### **A Dificuldade**: O erro `SCHEMA_CHANGE` ocorria com frequência quando arquivos Parquet de diferentes períodos apresentavam variações sutis de tipos (ex: `varchar` vs `timestamp`), paralisando o pipeline.

* ### **A Solução**: Implementação de **Schema Enforcement** (Imposição de Esquema). Criamos um catálogo de metadados centralizado no Airflow e passamos a injetar funções de **CAST** explícitas em todos os processos de registro de tabelas, garantindo que o Lakehouse operasse sob um contrato de dados rígido e previsível.

### A transição entre a camada Bronze (dados brutos tipados em Parquet) e as camadas subsequentes (Silver/Gold) enfrentou um cenário de falha crítica, aqui denominado "apocalipse de metadados", que motivou uma reavaliação completa da arquitetura proposta para o MVP.

### **O Conflito do Catálogo Gerenciado (Nessie vs. SQL)**
### Houve uma barreira técnica na forma como o SQL de criação de tabelas era interpretado pelo Dremio dentro de uma fonte versionada.
* ### **A Dificuldade**: Tentativas de utilizar a cláusula `LOCATION` no comando `CREATE TABLE` resultaram em erros de "argumento inválido", pois fontes gerenciadas pelo Nessie proíbem a definição manual do caminho físico dos dados.

### **Anatomia da Falha**
### A falha não foi causada por um único erro, mas pela convergência de três fatores técnicos distintos que resultaram na perda total da consistência do catálogo de dados:
* ### **Amnésia de Sessão e Desacoplamento de Contexto**: O componente de orquestração [`DataOpsManager`](https://github.com/gabrielsimas/rio-viagens-onibus-analytics/blob/master/dags/modules/dataops_manager.py) operava em um modelo de conexões efêmeras. Ao executar comandos de troca de contexto (`USE REFERENCE`) e criação de tabelas em sessões ODBC distintas, o sistema sofria de uma "amnésia" de estado, resultando na gravação de metadados diretamente na branch principal (`main`), ignorando o isolamento de desenvolvimento pretendido.

* ### **Falha de Persistência em Camada de Infraestrutura**: Devido a restrições de permissão no sistema de arquivos da VPS, o banco de dados de metadados (RocksDB) do catálogo Nessie falhou ao persistir dados no volume físico. Como consequência, o catálogo operava inteiramente em memória volátil, tornando-o resiliente a consultas em tempo de execução, mas vulnerável a qualquer reinicialização do serviço.

* ### **Divergência de Linhagem (Merge Inversion)**: A tentativa de sincronizar branches de desenvolvimento vazias com uma branch principal que continha dados órfãos gerou um estado de inconsistência no motor de catálogo. Isso causou operações de merge que, na prática, propagavam o estado "vazio" para as tabelas que deveriam ser povoadas.

### **Decisão Estratégica: Simplificação para Robustez**
### Diante da fragilidade apresentada pela sobreposição de ferramentas (Nessie e dbt) para um escopo de MVP, optou-se pela "**Lei da Parcimônia**" (Navalha de Ockham). A arquitetura foi simplificada para eliminar o catálogo intermediário, permitindo que o motor de consulta (Dremio) gerenciasse tabelas Iceberg diretamente no armazenamento em nuvem (GCS).
### A remoção do Nessie e do dbt reduziu a superfície de ataque de erros em 60%, eliminando a necessidade de gestão manual de sessões transacionais e garantindo que a persistência dos dados dependesse exclusivamente do storage de objetos, que possui alta disponibilidade nativa.